Inverse problem for the Lorenz system with exogenous input

Example Source: https://deepxde.readthedocs.io/en/latest/demos/pinn_inverse/lorenz.inverse.forced.html

In [15]:
import deepxde as dde
import numpy as np
import tensorflow as tf


In [16]:
# We also want to define our three unknown variables, 𝜎, 𝜌 and 𝛽
# which will now be called C1, C2, and C3, respectivly. These variables are given an initial guess of 1.0.

C1 = dde.Variable(1.0)
C2 = dde.Variable(1.0)
C3 = dde.Variable(1.0)

In [17]:
# Now we can begin by creating a TimeDomain class.

geom = dde.geometry.TimeDomain(0, 3)

In [18]:
def Lorenz_system(x, y):
    y1, y2, y3 = y[:, 0:1], y[:, 1:2], y[:, 2:]
    dy1_x = dde.grad.jacobian(y, x, i=0)
    dy2_x = dde.grad.jacobian(y, x, i=1)
    dy3_x = dde.grad.jacobian(y, x, i=2)
    return [
        dy1_x - C1 * (y2 - y1),
        dy2_x - y1 * (C2 - y3) + y2,
        dy3_x - y1 * y2 + C3 * y3,
    ]

In [19]:
def boundary(_, on_initial):
    return on_initial

In [20]:
ic1 = dde.icbc.IC(geom, lambda X: -8, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda X: 7, boundary, component=1)
ic3 = dde.icbc.IC(geom, lambda X: 27, boundary, component=2)

In [21]:
def gen_traindata():
    data = np.load("/Users/vassilis/Documents/GitHub/PFAS_PBK_models/PFAS Rainbow trout PBK/PINN/DeepXDE_implementation/example/Lorenz.npz")
    return data["t"], data["y"]

In [22]:
observe_t, ob_y = gen_traindata()
observe_y0 = dde.icbc.PointSetBC(observe_t, ob_y[:, 0:1], component=0)
observe_y1 = dde.icbc.PointSetBC(observe_t, ob_y[:, 1:2], component=1)
observe_y2 = dde.icbc.PointSetBC(observe_t, ob_y[:, 2:3], component=2)

In [23]:
observe_t

array([[0.        ],
       [0.01071738],
       [0.06430429],
       [0.1226244 ],
       [0.20389738],
       [0.2774039 ],
       [0.36605932],
       [0.46484636],
       [0.57747023],
       [0.74410941],
       [0.87815709],
       [1.01220478],
       [1.15004686],
       [1.28280759],
       [1.42510765],
       [1.58172756],
       [1.75397377],
       [1.89764233],
       [2.04923335],
       [2.18795955],
       [2.37650218],
       [2.53747737],
       [2.69922368],
       [2.86424061],
       [3.        ]])

In [24]:
# Now that the problem is fully setup, we define the PDE as:

data = dde.data.PDE(
    geom,
    Lorenz_system,
    [ic1, ic2, ic3, observe_y0, observe_y1, observe_y2],
    num_domain=400,
    num_boundary=2,
    anchors=observe_t,
)

# Where num_domain is the number of points inside the domain, and num_boundary is
# the number of points on the boundary. anchors are extra points beyond num_domain
# and num_boundary used for training. auxiliary_var_function is the interpolation
# function of 𝑓(𝑡)
# we defined above.

In [25]:
# Next, we choose the network. Here, we use a fully connected neural network of
# depth 4 (i.e., 3 hidden layers) and width 40:
net = dde.nn.FNN([1] + [40] * 3 + [3], "tanh", "Glorot uniform")
model = dde.Model(data, net)

In [26]:
external_trainable_variables =[C1, C2, C3]
# train adam
model.compile(
    "adam", lr=0.001, external_trainable_variables=external_trainable_variables
)


Compiling model...
'compile' took 0.002096 s



In [27]:
variable = dde.callbacks.VariableValue(
    external_trainable_variables, period=600, filename="variables.dat"
)
checkpointer = dde.callbacks.ModelCheckpoint(
    filepath= '/Users/vassilis/Documents/GitHub/PFAS_PBK_models/PFAS Rainbow trout PBK/PINN/DeepXDE_implementation/example/model.ckpt',
    verbose=1, save_better_only=True, period=1, monitor='train loss', disregard_previous_best=False)

TypeError: ModelCheckpoint.__init__() got an unexpected keyword argument 'disregard_previous_best'

In [ ]:
losshistory, train_state = model.train(iterations=10000, callbacks=[variable, checkpointer])

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

In [ ]:
model.state_dict()

In [ ]:
net = dde.nn.FNN([1] + [40] * 3 + [3], "tanh", "Glorot uniform")
model = dde.Model(data, net)

external_trainable_variables =[C1, C2, C3]

# train adam
model.compile(
    "adam", lr=0.001, external_trainable_variables=external_trainable_variables
)
model.restore('/Users/vassilis/Documents/GitHub/PFAS_PBK_models/PFAS Rainbow trout PBK/PINN/DeepXDE_implementation/example/model.ckpt-10000.ckpt', verbose=1)

In [ ]:
# model.restore('/Users/vassilis/Documents/GitHub/PFAS_PBK_models/PFAS Rainbow trout PBK/PINN/DeepXDE_implementation/example/model.ckpt-10000.ckpt', verbose=1)

In [ ]:
model.predict(observe_t)

In [ ]:
observe_t

In [ ]:
model.state_dict()

In [ ]:
net = dde.nn.FNN([1] + [40] * 3 + [3], "tanh", "Glorot uniform")
model = dde.Model(data, net)

external_trainable_variables =[C1, C2, C3]

# train adam
model.compile(
    "adam", lr=0.001, external_trainable_variables=external_trainable_variables)
    
losshistory, train_state = model.train(iterations=2000, callbacks=[variable, checkpointer], 
                                      model_restore_path= '/Users/vassilis/Documents/GitHub/PFAS_PBK_models/PFAS Rainbow trout PBK/PINN/DeepXDE_implementation/example/model.ckpt-10000.ckpt')

In [ ]:
model.state_dict()